In [ ]:
import hpelm
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

from keras.layers.core import Dense, Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator

import plotly
from plotly.graph_objs import Layout, Figure, Marker
import time
CNN_EPOCH = 10
image_shape=(224,224,3)
NUM_CLASS = 2


import os
import cv2
from PIL import Image

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# metric
from keras.metrics import binary_crossentropy
# optimization method (Stochastic Gradient Descent (SGD))

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

#Train IELM and evaluating training accuracy
no_classes = 2
error = 0.1
loss_function = "binary_cross_entropy" #"mean_squared_error"  #It can be mean_absolute_error also
activation_function = "tanh"
import tensorflow as tf


In [ ]:
def loading_data(train_path,test_path):

    # Define the data transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to (224, 224)
        transforms.ToTensor(),  # Convert images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    ])
 

    train_dataset = ImageFolder(train_path, transform=transform)
    traindata_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

    test_dataset = ImageFolder(test_path, transform=transform)
    testdata_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
    return traindata_loader,testdata_loader


In [ ]:

def load_model():
    #loading of model
    device=torch.device("cuda")  
    
    base_model = models.alexnet(pretrained=True)
    base_model = torch.nn.Sequential(*list(base_model.children())[:-1])
    base_model=base_model.to(device)
    base_model.eval()
    return base_model
#training features extraction
def training_feature_extraction(device, traindata_loader,resnet_model):

    x_training_features = []
    y_train_labels = []
    with torch.no_grad():
        for images, labels in traindata_loader:        
            images = images.to(device)
            labels = labels.to(device)
            # Extract features using VGG16
            with torch.no_grad():
                features = resnet_model(images)   
            
            x_training_features.append(features)
            y_train_labels.append(labels)
    x_training_features = torch.cat(x_training_features, dim=0)
    y_train_labels = torch.cat(y_train_labels, dim=0)

    x_training_features.to(device)
    y_train_labels.to(device)
    return x_training_features,y_train_labels

def testing_feature_extratcion(device,testdata_loader,vgg_model):
    x_testing_features = []
    y_testing_labels = []
    with torch.no_grad():
        for images, labels in testdata_loader:
            images = images.to(device)
            labels = labels.to(device)
            # Extract features using VGG19
            with torch.no_grad():
                features = vgg_model(images) 
            x_testing_features.append(vgg_model(images))
            y_testing_labels.append(labels)
    x_testing_features = torch.cat(x_testing_features, dim=0)
    y_testing_labels = torch.cat(y_testing_labels, dim=0)

    x_testing_features.to(device)
    y_testing_labels.to(device)
    return x_testing_features,y_testing_labels


In [ ]:
def train_features_conversion(x_training_features,y_train_labels):

    # Convert the extracted features to numpy arrays

    device=torch.device("cuda")
    x_training_features_np = x_training_features.detach().cpu().numpy()  
    flattened_features_train = x_training_features_np.reshape(x_training_features_np.shape[0], -1)
    train_features_tensor = torch.tensor(flattened_features_train, dtype=torch.float32).to(device)
    
    y_train_array= y_train_labels.cpu().detach().numpy()
    y_train_array_encoded = np_utils.to_categorical(y_train_array,no_classes)
    y_train_labels_encoded=torch.tensor(y_train_array_encoded,dtype=torch.float32)
    y_train_labels_encoded=y_train_labels_encoded.to(device)    
    return train_features_tensor,y_train_labels_encoded

def test_features_conversion(x_testing_features,y_testing_labels):
    # Convert the extracted features to numpy arrays
    device=torch.device("cuda")
    x_testing_features_np = x_testing_features.detach().cpu().numpy()
    
    flattened_features_test = x_testing_features_np.reshape(x_testing_features_np.shape[0], -1)
    test_features_tensor = torch.tensor(flattened_features_test,dtype=torch.float32).to(device)
    
    y_test_array= y_testing_labels.cpu().detach().numpy()
    y_test_array_encoded = np_utils.to_categorical(y_test_array,no_classes)
    y_test_labels_encoded=torch.tensor(y_test_array_encoded,dtype=torch.float32)
    y_test_labels_encoded=y_test_labels_encoded.to(device)
    return test_features_tensor, y_test_labels_encoded

In [ ]:
device=torch.device("cuda")
train_path="D:/ALEXELM/Dehazed_Images_3400/Dehazed_Images/train/"
test_path="D:/ALEXELM/Dehazed_Images_3400/Dehazed_Images/test/"
traindata_loader,testdata_loader=loading_data(train_path,test_path)
model=load_model()


In [ ]:
x_training_features,y_train_labels=training_feature_extraction(device,traindata_loader,model)
train_features_tensor,y_train_labels_encoded=train_features_conversion(x_training_features,y_train_labels)


In [ ]:
torch.save(train_features_tensor,"alexnet_dehazed_train_features.pth")
torch.save(y_train_labels_encoded,"alexnet_dehazed_train_labels.pth")

In [ ]:
x_testing_features,y_testing_labels=testing_feature_extratcion(device,testdata_loader,model)
test_features_tensor, y_test_labels_encoded=test_features_conversion(x_testing_features,y_testing_labels)
torch.save(test_features_tensor,"alexnet_dehazed_test_features.pth")
torch.save(y_test_labels_encoded,"alexnet_dehazed_test_labels.pth")

In [ ]:
import torch
device= torch.device("cuda")
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used for training.")
else:
    print("CUDA is not available. CPU will be used for training.")


In [ ]:
#code for selection of best parameters
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
def best_parameter():
# Wrap the model using KerasClassifier
    keras_model = KerasClassifier(build_fn=Alexnet, verbose=0)

    # Define the parameter grid
    param_grid = {
        'optimizer': ['adam', 'rmsprop','SGD'],
        'learning_rate': [0.001, 0.01, 0.1],
        # Add more hyperparameters to tune as needed
    }


    # Instantiate GridSearchCV
    grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2)

    #y_train_max= np.array([np.argmax(r) for r in y_train])
    # Fit the grid search to the data
    grid_result = grid_search.fit(X_train, y_train)

    # Print the best parameters found
    print("Best Parameters:", grid_result.best_params_)

    # Get the best model
    best_model = grid_result.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
y_test_max= np.array([np.argmax(r) for r in y_test])
accuracy = accuracy_score(y_test_max, y_pred)
print("Test Set Accuracy:", accuracy)
device=torch.device("cuda") 
train_history, acc ,alexmodel= train_model(base_model, traindata_loader, testdata_loader, epochs =10, batch_size = 32)

In [ ]:
#visualisation of features
from keras.models import load_model,Model
#from keras.preprocessing.image import load_images_and_labels
from tensorflow.keras.preprocessing.image import load_img,img_to_array

# Load an image from a file
#img = load_img('path_to_image.jpg', target_size=(224, 224))
vm=load_model("alex_model_10.h5")

cnnModel=Sequential()
cnnModel=Model(inputs=vm.inputs, outputs=vm.layers[6].output)
cnnModel.summary()
#for i in range(10):
print(vm.layers[0].output)
print(vm.layers[2].output)
print(vm.layers[4].output)
print(vm.layers[5].output)
print(vm.layers[6].output)
#print(vm.layers[8].output)

#vm.layers[1].
img=load_img("D:/ALEXELM/Dehazed_Images_3400/Dehazed_Images/train/busy/(35).jpg",target_size=(227,227,3))
img_arr=img_to_array(img)
img_arr=np.expand_dims(img_arr,axis=0)

feature_maps=cnnModel.predict(img_arr)
square,ix=4,1
from matplotlib import pyplot as plt
for _ in range(square):
    for _ in range(square):
        ax=plt.subplot(square,square,ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(feature_maps[0,:,:,ix-1],cmap='pink')
        ix+=1
plt.show()

In [ ]:
x_training_features=torch.load("alexnet_dehazed_train_features.pth")
y_train_labels=torch.load("alexnet_dehazed_train_labels.pth")


In [ ]:
x_testing_features=torch.load("alexnet_PUC_test_features.pth")
y_testing_labels=torch.load("alexnet_PUC_test_labels.pth")


In [ ]:

y_train_labels_reshaped=np.argmax(y_train_labels.detach().cpu().numpy(),axis=1)


In [ ]:
y_test_labels_reshaped=np.argmax(y_testing_labels.detach().cpu().numpy(),axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import numpy as np

def featureselection_rf(x_training_features,y_train_labels_reshaped):
    # Train a Random Forest classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(x_training_features.cpu().numpy(), y_train_labels_reshaped)

    # Get feature importance scores
    importance_scores = rf.feature_importances_

    # Sort features based on importance scores
    sorted_indices = np.argsort(importance_scores)[::-1]


    # Select top k features (e.g., top 2 features)
    k = 10
    selected_features = sorted_indices[:k]

    # Print selected features and their importance scores
    for i in selected_features:
        print(f"Feature {i}: Importance Score = {importance_scores[i]}")

    
    return selected_features


In [ ]:
#***************ML_ELM********************
class ML_ELM_:
    def __init__(self, l1, l2, l3,l4):
        self.l1 = l1
        self.l2 = l2
        self.l3 = l3
        self.l4 = l4
        pass

        def hidden_nodes(X, input_weights, biases):
            from numpy import dot
            def relu(x):
                from numpy import maximum
                return maximum(x, 0, x)

            G = dot(X, input_weights)
            G = G + biases
            H = relu(G)
            return H

        self.hidden_nodes = hidden_nodes

        def ELM_hidden(train_in, input_weights):
            biases = 0
            hidden = hidden_nodes(train_in, input_weights, biases)

            return hidden

        self.ELM_hidden = ELM_hidden

    def fit(self, train_in_, train_out):
        def ELM_weights(train_in, hidden_size):
            from scipy.linalg import pinv
            from numpy import random
            from numpy import dot
            from numpy import transpose

            input_size = train_in.shape[1]
            hidden_size = int(hidden_size)
            input_weights = random.normal(size=[input_size, hidden_size])
            biases = random.normal(size=[hidden_size])

            output_weights = dot(pinv(self.hidden_nodes(train_in, input_weights, biases)), train_in)
            transposed_weights = transpose(output_weights)

            return transposed_weights

        def ELM_train(train_in, train_out):
            from scipy.linalg import pinv
            from numpy import dot

            output_weights = dot(pinv(train_in), train_out)

            return output_weights

        self.layers = [i for i in list([self.l1, self.l2, self.l3, self.l4]) if i != 0]
        #self.layers = [i for i in list([self.l1, self.l2, self.l3]) if i != 0]
        input_layers_tr = [train_in_]

        self.weights = []

        for i, layer in enumerate(self.layers):
            self.weights += [ELM_weights(input_layers_tr[i], layer)]
            input_layers_tr += [self.ELM_hidden(input_layers_tr[i], self.weights[i])]

        self.weights += [ELM_train(input_layers_tr[-1], train_out)]
        return self

    def predict(self, train_in_):
        def predict(input, output_weights):
            from numpy import dot
            out = dot(input, output_weights)
            return out

        def ELM_predict(input, output_weights):
            prediction = predict(input, output_weights)
            return prediction

        input_layers_tr = [train_in_]
        for i, layer in enumerate(self.layers):
            input_layers_tr += [self.ELM_hidden(input_layers_tr[i], self.weights[i])]
        output_train_pred = ELM_predict(input_layers_tr[-1], self.weights[-1])
        #print(output_train_pred)
        return output_train_pred